# 03.00 Document stores
Instead of using a ORM like SQLalachemy (or directly tables via SQL), we can store data as documents.

In [1]:
data = '''{
"title" : "Tux states a truth!",
"text" : "Sealions and penguins will never be friends!",
"author" : "Tux",
 "comments" : [{"author" : "Sealion",
 "text":"This is not true, we had fish together last Sunday."},
 {"author" : "Crabby",
 "text" : "I so agree, these sealions think they can get away with anything."}]
 }'''

In [2]:
import json

In [3]:
d = json.loads(data)

In [4]:
d['title']

'Tux states a truth!'

In [5]:
d['author']

'Tux'

In [6]:
d['comments']

[{'author': 'Sealion',
  'text': 'This is not true, we had fish together last Sunday.'},
 {'author': 'Crabby',
  'text': 'I so agree, these sealions think they can get away with anything.'}]

In [8]:
[{"name" : "Pikachu",
 "category" : "mouse"},
{"name" : "Bulbasaur",
"category" : "seed"},
{"name" : "Charmander",
"category" : "lizard"}]

[{'name': 'Pikachu', 'category': 'mouse'},
 {'name': 'Bulbasaur', 'category': 'seed'},
 {'name': 'Charmander', 'category': 'lizard'}]

## 3.01 Connecting to MongoDB
For the flask plugin take a look at https://flask-pymongo.readthedocs.io/en/latest/

In [9]:
import pymongo

In [10]:
client = pymongo.MongoClient()

In [11]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [15]:
# create or get a database called cs6
db = client['cs6']

In [16]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'cs6')

## 3.02 Inserting & updating data
Let's say we want to add a new document with Pikachu's details

In [27]:
db.pokemon.insert_one({'name' : 'Pikachu', 'category' : 'mouse'})

To retrieve it, we can use `find` or `find_one`

In [28]:
list(db.pokemon.find())

[{'_id': ObjectId('5dd416c5fa615e70c81f5f2b'),
  'name': 'Pikachu',
  'category': 'mouse'}]

To insert another pokemon, we can also use a python dictionary!

In [29]:
p = {'name' : 'Bulbasaur', 'category' : 'seed'}

In [30]:
db.pokemon.insert_one(p)

In [31]:
list(db.pokemon.find())

[{'_id': ObjectId('5dd416c5fa615e70c81f5f2b'),
  'name': 'Pikachu',
  'category': 'mouse'},
 {'_id': ObjectId('5dd416c7fa615e70c81f5f2c'),
  'name': 'Bulbasaur',
  'category': 'seed'}]

What about objects?

Unfortunately, they can't be directly stored - again a mapping here not ORM, but ODM = Object Document Mapping is required. 
Ming is e.g. such a mapper https://ming.readthedocs.io/en/latest/


The easiest solution though is to serialize the object as dict or JSON!

In [39]:
class Pokemon:
    
    def __init__(self, name, category):
        self.name = name
        self.category = category
        
    def serialize(self):
        return {'name' : self.name, 'category' : self.category}

In [40]:
p = Pokemon('Charmander', 'lizard')

In [41]:
db.pokemon.insert_one(p.serialize())

In [42]:
list(db.pokemon.find())

[{'_id': ObjectId('5dd416c5fa615e70c81f5f2b'),
  'name': 'Pikachu',
  'category': 'mouse'},
 {'_id': ObjectId('5dd416c7fa615e70c81f5f2c'),
  'name': 'Bulbasaur',
  'category': 'seed'},
 {'_id': ObjectId('5dd41893fa615e70c81f5f2d'),
  'name': 'Charmander',
  'category': 'lizard'}]

What if we want to add fields to a document? Thankfully in MongoDB this is very easy! Unlike for a relational database, we don't have to change the schema!

In [43]:
db.pokemon.update_one({'name' : 'Pikachu'}, {'$set' : {'height' : 1.04, 'weight' : 13.2}})

In [44]:
db.pokemon.find_one({'name' : 'Pikachu'})

{'_id': ObjectId('5dd416c5fa615e70c81f5f2b'),
 'name': 'Pikachu',
 'category': 'mouse',
 'height': 1.04,
 'weight': 13.2}

Of course there are a lot more operations in MongoDB, https://api.mongodb.com/python/current/ is the primary resource for this and how to use the PyMongo adapter.

To reset this notebook, use

In [45]:
# use this to clear a collection
db.pokemon.drop()